## The Geometry of Bézier splines

> **_Tip:_** Launch live version of this tutorial: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/morphomatics/morphomatics.github.io/HEAD?filepath=docs%2Ftutorials%2Ftutorial_bezierfol.ipynb)

It happens frequently that one is interested in the statistical analysis of curves. For example, in order to analyze longitudinal data (e.g., measurements from several subjects at different points in time in a medial study) statistically, **[hierarchical models](https://en.wikipedia.org/wiki/Multilevel_model)** (see also **[this paper](https://arxiv.org/abs/2111.15371)**) are widely used. By first approximating _subject-wise_ trends (the individual level) and _subsequently_ computing the statistics of these curves (the group level), they account for the fact that intra-subject measurements are highly correlated while inter-subject measurements are not.

In order to analyze Bézier splines thorugh a Riemannian manifold $M$ statistically, we need a Riemannian structure on the _space of Bézier curves_ that go through $M$. Such a structure is implemented in Morphomatics; see **[A Nonlinear Hierarchical Model for Longitudinal Data on Manifolds](https://arxiv.org/abs/2202.01180)** for the definitions. In this tutorial we show how it can be used to compute the mean of two Bézier splines in the 2-sphere $S^2$.

In [1]:
%env JAX_PLATFORM_NAME=cpu
import jax
import jax.numpy as jnp

from morphomatics.geom.BezierSpline import BezierSpline
from morphomatics.manifold.Bezierfold import Bezierfold
from morphomatics.manifold.Sphere import Sphere

env: JAX_PLATFORM_NAME=cpu


We start by initializing $S^2$ and the manifold of Bézier splines (''Bézierfold'') consisting of splines in $S^2$ with two quadratic segments; we also initialize the Riemannian structure on the Bézierfold.

In [2]:
M = Sphere()

B = Bezierfold(M, 2, 2)
B.initFunctionalBasedStructure()

The following lines create two splines as data.

In [3]:
North = jnp.array([0., 0., 1.])
South = jnp.array([0., 0., -1.])

p = jnp.array([1., 0., 0.])
o = jnp.array([1/jnp.sqrt(2), 1/jnp.sqrt(2), 0.])
om = M.connec.exp(o, jnp.array([0, 0, -.25]))
op = M.connec.exp(o, jnp.array([0, 0, .25]))
q = jnp.array([0, 1, 0.])

B1 = BezierSpline(M, [jnp.stack((p, om, o)), jnp.stack((o, op, q))])

z = M.connec.geopoint(o, North, .5)

p1 = jnp.array([1., 0., 0.])
o1 = M.connec.geopoint(p, z, .5)
om1 = M.connec.geopoint(p, z, .4)
op1 = M.connec.geopoint(p, z, .6)
q1 = z
B2 = BezierSpline(M, [jnp.stack((p1, om1, o1)), jnp.stack((o1, op1, q1))])

Now, we can compute the mean of the two splines and plot the results. Note that the solution is computed iteratively. Therefore, the parameters _n_, _delta_, and _nsteps_ can be used to balance speed and accuracy: The larger _n_ and _nsteps_ and the smaller _delta_, the more accurate (but the slower) the computation will be. Finally, we visualize the two splines and their mean.

In [ ]:
#mean = B.metric.mean([B1, B2], n=3, delta=1e-5, nsteps=5)[0]
mean = B.metric.mean([B1, B2], n=3, delta=1e-5, nsteps=1)[0]

import numpy as np
import pyvista as pv

# Plotting

# create a tubular surface interpolating points
def tube_mesh(pts, interval=(0,2)):
    n = len(pts)-1
    curve = pv.PolyData(pts)
    curve["time"] = np.linspace(*interval, curve.n_points)
    curve.lines = np.c_[np.full(n,2), np.arange(n), np.arange(n)+1]
    curve.tube(radius=0.01, inplace=True)
    return curve

pl = pv.Plotter(notebook=True)
tube1 = tube_mesh(np.asarray(jax.vmap(B1.eval)(jnp.linspace(0., 2., num=100))))
tube2 = tube_mesh(np.asarray(jax.vmap(B2.eval)(jnp.linspace(0., 2., num=100))))

pl.add_mesh(tube_mesh(np.asarray(jax.vmap(mean.eval)(jnp.linspace(0., 2., num=100)))), color='red')
pl.add_mesh(tube1)
pl.add_mesh(tube2)

pl.add_mesh(pv.Sphere(1))

pl.show_axes()
pl.show(jupyter_backend='static')

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
